In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
'''
# Cargo de Google Drive los archivos

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'
downloaded = drive.CreateFile({'id': id_train_transaction})
downloaded.GetContentFile('train_transaction.csv')

id_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'
downloaded = drive.CreateFile({'id': id_train_identity})
downloaded.GetContentFile('train_identity.csv')

id_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'
downloaded = drive.CreateFile({'id': id_test_transaction})
downloaded.GetContentFile('test_transaction.csv')

id_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'
downloaded = drive.CreateFile({'id': id_test_identity})
downloaded.GetContentFile('test_identity.csv')'''

"\n# Cargo de Google Drive los archivos\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\nid_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'\ndownloaded = drive.CreateFile({'id': id_train_transaction})\ndownloaded.GetContentFile('train_transaction.csv')\n\nid_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'\ndownloaded = drive.CreateFile({'id': id_train_identity})\ndownloaded.GetContentFile('train_identity.csv')\n\nid_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'\ndownloaded = drive.CreateFile({'id': id_test_transaction})\ndownloaded.GetContentFile('test_transaction.csv')\n\nid_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'\ndownloaded = drive.CreateFile({'id': id_test_identity})\ndownloaded.GetContentFile('tes

In [ ]:
train_identity = pd.read_csv('./ieee-fraud-detection/train_identity.csv')

train_transaction = pd.read_csv('./ieee-fraud-detection/train_transaction.csv')

# Analizo los datasets

Creo los datasets encodeardos, limpios y escalados que voy a utilizar para entrenar y validar el modelo.

In [ ]:
train_identity

In [ ]:
train_transaction

> ***Observo que train_transaction tiene una columna de TransactionDT.***



Esta es una columna es de tipo datetime y representa el tiempo transcurrido desde un instante de referencia (desconocido). 

Esto significa que debe permanecer ordenada para evitar time-travel. 

In [ ]:
# Verifico que esten ordenado los registros en el tiempo

list(train_transaction.TransactionDT) == sorted(train_transaction.TransactionDT, reverse=False)

## Busco el dataset completo con el que voy a trabajar

Uno los data sets de trainning que nos dan.

In [ ]:
all_data = train_transaction.merge(train_identity, on='TransactionID', how='outer')

In [ ]:
import gc

# Libero memoria del sistema
del(train_transaction)
del(train_identity)

# using the garbage collector to manually collect the garbage
gc.collect()

Este truco lo saque de Kaggle para administrar la memoria.

In [ ]:
display(all_data)

Verifico que mantenga el orden segun TransactionDT

In [ ]:
all_data.TransactionDT.hasnans

In [ ]:
list(all_data.TransactionDT) == sorted(all_data.TransactionDT, reverse=False)

## Tomo 80 columnas del DataSet

Segun el modelo baseline de Regresion Logistica, tomo las 80 features mas importantes

In [ ]:
top80features = ['C14',
 'C2',
 'C11',
 'C13',
 'C7',
 'C3',
 'V192',
 'V222',
 'V179',
 'DeviceType',
 'V207',
 'V221',
 'C5',
 'V73',
 'V259',
 'V48',
 'M7',
 'V227',
 'V267',
 'card6',
 'V269',
 'V189',
 'V49',
 'ProductCD',
 'V147',
 'M1',
 'V18',
 'V257',
 'V200',
 'id_15',
 'V50',
 'V171',
 'V149',
 'V139',
 'D8',
 'id_29',
 'V208',
 'V323',
 'C8',
 'V182',
 'V210',
 'D3',
 'M6',
 'V188',
 'V229',
 'V33',
 'V71',
 'V285',
 'V153',
 'V58',
 'id_17',
 'V44',
 'TransactionDT',
 'M8',
 'V268',
 'V29',
 'V246',
 'M4',
 'V13',
 'V255',
 'V174',
 'V195',
 'M3',
 'V82',
 'V21',
 'V185',
 'V203',
 'V83',
 'V184',
 'V176',
 'id_36',
 'V294',
 'V168',
 'V332',
 'V175',
 'V333',
 'V225',
 'id_35',
 'V292',
 'V326']

In [ ]:
features_adicionales = ['id_31', 'id_33', 'DeviceType', 'DeviceInfo', 'isFraud', 'TransactionDT' ]

for feat in features_adicionales:
    if (not feat in top80features):
        top80features.append(feat)

In [ ]:
all_data = all_data[top80features]

display(all_data)

## Extraigo los target y features del dataset y creo los sets de trainning y testing.

Creo `data_train` y `data_test` para el set de `data` y

`label_train`, `label_test` para el set de `target`.

> Nota: El target es `isFraud`

### Particiono la data como train_set y test_set

Tomo un 20% de la data como `test_set` y el restante para `train_set` del modelo.

Me baso en TransactionDT como separador de registros.

In [ ]:
labels = all_data.isFraud

In [ ]:
# Tomo el 80% 
ochenta_porciento = int(all_data['TransactionDT'].size * 0.8)

In [ ]:
data_train = all_data.iloc[: ochenta_porciento]

data_validation = all_data.iloc[ochenta_porciento:]

In [ ]:
label_train = labels.iloc[:ochenta_porciento]

label_validation = labels.iloc[ochenta_porciento:]

## Limpio la data

Para los features categoricos reemplazo los 'nans' por 'desconocido' y para los features numericos los reemplazo por su mean de cada columna.

> Nota: Para el validation test reemplazo los nans por los means correspondientes a las columnas del trainning_set.
>
>  No se vuelven a calcular los means por columnas.

In [ ]:
# Busco los promedios de cada columna numerica para el trainning_set

mean_values={}

for i in data_train.columns:
  if (str(data_train[i].dtype) != 'object'):
    mean_values[i] = data_train[i].mean()

In [ ]:
# Limpio la data

def limpiar_dataset(dataset, mean_values):
  
  for i in dataset.columns:

    if (str(dataset[i].dtype) != 'object'):
      dataset[i].fillna(mean_values[i], inplace=True)
    
    else:
      dataset[i].fillna('desconocido',inplace=True)

In [ ]:
limpiar_dataset(data_train, mean_values)
display(data_train)

In [ ]:
data_validation

In [ ]:
limpiar_dataset(data_validation, mean_values)
display(data_validation)

In [ ]:
data_validation.reset_index(inplace=True, drop=True)
display(data_validation)

## Limpio algunos features que tienen demasidos valores unicos

Categorizamos algunos features que tienen demasiados valores unicos para pòder trabajar mejor en el modelo.

Utilizo CountVectorizer para DeviceInfo, id_31 y id_33 con un maximo de 10 palabras mas comunes

In [ ]:
def obtener_columnas_categoricas(data_set):

  columnas_categoricas = []

  for col in data_set.columns:
        if (str(data_set[col].dtype) == 'object'):
            columnas_categoricas.append(col)

  return columnas_categoricas

### ID 31

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

coun_vect_id31 = CountVectorizer(min_df=10, max_features=15, lowercase=True,  stop_words= ['desconocido','11'])

count_vect_id31_entrenado = coun_vect_id31.fit(data_train.id_31)

In [ ]:
count_vec_id_31_data_train =   pd.DataFrame(count_vect_id31_entrenado.transform(data_train.id_31).toarray(), columns = coun_vect_id31.get_feature_names())

data_train.drop(columns='id_31',inplace=True)
data_train = pd.concat([data_train, count_vec_id_31_data_train],axis=1)

display(data_train)

In [ ]:
count_vec_id_31_data_validation =   pd.DataFrame(count_vect_id31_entrenado.transform(data_validation.id_31).toarray(), columns = coun_vect_id31.get_feature_names())

data_validation.drop(columns='id_31',inplace=True)
data_validation = pd.concat([data_validation, count_vec_id_31_data_validation],axis=1)

display(data_validation)

### ID 33

In [ ]:
coun_vect_id33 = CountVectorizer(min_df=10, max_features=15,lowercase=True,  stop_words= ['desconocido','11'])

count_vect_id33_entrenado = coun_vect_id33.fit(data_train.id_33)

In [ ]:
count_vec_id_33_data_train =  pd.DataFrame(count_vect_id33_entrenado.transform(data_train.id_33).toarray(), columns = coun_vect_id33.get_feature_names())

data_train.drop(columns='id_33',inplace=True)
data_train = pd.concat([data_train, count_vec_id_33_data_train],axis=1)

display(data_train)

In [ ]:
count_vec_id_33_data_validation =   pd.DataFrame(count_vect_id33_entrenado.transform(data_validation.id_33).toarray(), columns = coun_vect_id33.get_feature_names())

data_validation.drop(columns='id_33',inplace=True)
data_validation = pd.concat([data_validation, count_vec_id_33_data_validation],axis=1)

display(data_validation)

### DeviceInfo

In [ ]:
coun_vect_di = CountVectorizer(min_df=10, max_features=15, lowercase=True, stop_words= ['desconocido','11'])
count_vect_deviceInfo_entrenado = coun_vect_di.fit(data_train.DeviceInfo)

In [ ]:
count_vec_id_dI_data_train =   pd.DataFrame(count_vect_deviceInfo_entrenado.transform(data_train.DeviceInfo).toarray(), columns = coun_vect_di.get_feature_names())

data_train.drop(columns='DeviceInfo',inplace=True)
data_train = pd.concat([data_train, count_vec_id_dI_data_train],axis=1)

display(data_train)

In [ ]:
count_vec_id_dI_data_validation =   pd.DataFrame(count_vect_deviceInfo_entrenado.transform(data_validation.DeviceInfo).toarray(), columns = coun_vect_di.get_feature_names())

data_validation.drop(columns='DeviceInfo',inplace=True)
data_validation = pd.concat([data_validation, count_vec_id_dI_data_validation],axis=1)

display(data_validation)

### Verifico los nuevos features categoricos

In [ ]:
data_validation[obtener_columnas_categoricas(data_validation)].nunique()

In [ ]:
data_train[obtener_columnas_categoricas(data_train)].nunique()

## Aplico Mean Encoding a data_train y luego a validation_train

Guardo los encodings para los features categoricos en un diccionario para luego asignarselos al set de test.

In [ ]:
mean_encoding_columns = dict()

for col in data_train.columns:
    
    if(str(data_train[col].dtype) == 'object'):
       mean_encoding_columns[col]= data_train.groupby(col).isFraud.mean().to_dict()

In [ ]:
mean_encoding_columns

In [ ]:
def aplicar_mean_encoding(data_set, mean_encoding):
    
    for col in data_set.columns:
        if (str(data_set[col].dtype) == 'object'):
            data_set[col] = data_set[col].map(mean_encoding[col])

In [ ]:
# Aplico el encoder al data_train
aplicar_mean_encoding(data_train, mean_encoding_columns)
display(data_train)

In [ ]:
# Aplico el encoder al data_validation
aplicar_mean_encoding(data_validation, mean_encoding_columns)
display(data_validation)

## Aplico Scalling a la data para mejorar el modelo

In [ ]:
data_train = data_train.drop(columns=['isFraud'])

feature_names = data_train.columns # Los guardo para el punto extra

In [ ]:
data_validation = data_validation.drop(columns=['isFraud'])

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(data_train)

data_train = scaler.fit_transform(data_train)
data_validation = scaler.transform(data_validation)

In [ ]:
data_train

# Creo el Modelo de Random Forest y lo entreno

Teniendo la data ordenada, creo el modelo de Random Forest y lo entreno con `data_train` y `label_train`

In [ ]:
model = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
model.fit(data_train, label_train)

# Metricas

## Aucroc Score

In [ ]:
roc_auc_score(label_train, model.predict(data_train))

In [ ]:
roc_auc_score(label_validation, model.predict(data_validation))

# Busqueda de Hiper-parametros

Busco hiper-parametros para `XGBoost` utilizando `GridSearchCV`

In [ ]:
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV

[Hiper parametros de XGBooster](https://towardsdatascience.com/a-guide-to-xgboost-hyperparameters-87980c7f44a9)

In [ ]:
hiper_parametros = dict(objective =['reg:squarederror'],
                        colsample_bytree = [0.3], 
                        learning_rate = [0.1],
                        alpha = [10],
                        max_depth = [4,8,16],
                        n_estimators = [4,8,16,32] )

In [ ]:
gridSearch = GridSearchCV(model, hiper_parametros, cv=5, n_jobs=-1, scoring='roc_auc')

In [ ]:
search = gridSearch.fit(data_train, label_train)

## Busco los mejores coeficientes

In [ ]:
search.best_params_

In [ ]:
search.best_estimator_

# XGBoost con los mejores hiper-parametros obtenidos 

Usando XGBoost obtengo como Hiper-Parametros obtenidos



| Hiper-Parametro   |  Mejor Valor |  
|:---:|:---:|
|  alpha | 10  |  
|  colsample_bytree |  0.3  |
|  learning_rate |  0.1  |
|  max_depth |  16|
|  n_estimators |  32 |
|  objective |  reg:squarederror |

> Nota: El gridSearch hecho no es completamente representativo ya que XGBoost es muy pesado como para correr muchas pruebas, por eso los hiperparametros no cambian demasiado con respecto a los originalmente tomados.


Vuelvo a plantear XGBoost para ver como mejora el modelo.

In [ ]:
model = search.best_estimator_

model.fit(data_train, label_train)

## Guardo el modelo final

In [ ]:
import joblib

joblib.dump(model, "random_forest_model.pkl")

In [ ]:
roc_auc_score(label_train, model.predict(data_train))

In [ ]:
roc_auc_score(label_validation, model.predict(data_validation))

# Score en la competencia

Al comparar con RandomForest, se observa que XGBoost tiene una auc_roc score con el validation set de `0.85197`, mientras que RandomForest nos dio un score de `0.847`. 

Por lo tanto determino el valor que tiene en la competencia, por tratarse del mejor modelo entre los dos.

## Cargo los archivos de testing

In [ ]:
test_identity = pd.read_csv('./ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('./ieee-fraud-detection/test_transaction.csv')

In [ ]:
test_identity.columns = test_identity.columns.str.replace('id-','id_')

In [ ]:
test_data = test_transaction.merge(test_identity, on='TransactionID', how='outer')

del(test_identity)
del(test_transaction)

gc.collect()

In [ ]:
# Verifico que este ordenada
list(test_data.TransactionDT) == sorted(test_data.TransactionDT, reverse=False)

## Tomo las features de interes

In [ ]:
transactionID = test_data.TransactionID
top80features.remove('isFraud')
test_data = test_data[top80features]

## Limpio la data

In [ ]:
limpiar_dataset(test_data, mean_values)

## Aplico vectorizer

In [ ]:
count_vec_id_31_test_set =   pd.DataFrame(count_vect_id31_entrenado.transform(test_data.id_31).toarray(), columns = coun_vect_id31.get_feature_names())
count_vec_id_33_test_set =   pd.DataFrame(count_vect_id33_entrenado.transform(test_data.id_33).toarray(), columns = coun_vect_id33.get_feature_names())
count_vec_id_dI_test_set =   pd.DataFrame(count_vect_deviceInfo_entrenado.transform(test_data.DeviceInfo).toarray(), columns = coun_vect_di.get_feature_names())

In [ ]:
test_data.drop(columns=['id_31','id_33','DeviceInfo'],inplace=True)

test_data = pd.concat([test_data, count_vec_id_31_test_set],axis=1)
test_data = pd.concat([test_data, count_vec_id_33_test_set],axis=1)
test_data = pd.concat([test_data, count_vec_id_dI_test_set],axis=1)

## Aplico Mean Encoding

In [ ]:
# Aplico el encoder al data_train
aplicar_mean_encoding(test_data, mean_encoding_columns)
display(data_train)

## Aplico scaler

In [ ]:
test_data = scaler.transform(test_data)

## Hago las predicciones

In [ ]:
predictions = model.predict(test_data)

In [ ]:
sample_submition = pd.DataFrame()
sample_submition['TransactionID'] = list(transactionID)
sample_submition['isFraud'] = predictions

sample_submition.set_index('TransactionID', inplace=True)
display(sample_submition)

In [ ]:
sample_submition.to_csv('sample_submition_XGB.csv')

## Conclusion

El auc_score obtenido de la competencia con el modelo de XGBoost es de `0.880579`. Es una mejora del modelo de Linear Regression el cual dio 0.857682

# Puntos Extra

Graficar la importancia de features para algun modelo de la parte III. ¿Que tanto se parece a los features importantes de la parte II?

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

plot_importance(model, max_num_features=10, title='Top 10 Features')
pyplot.show()

In [ ]:
feature_names

In [ ]:
importancia_features = [(feature, coeff) for feature, coeff in zip(feature_names,model.feature_importances_)]
print(importancia_features)

In [ ]:
top10 = sorted(importancia_features, key = lambda x: -abs(x[1]) )[:10]
print(top10)

In [ ]:
feature_names = [x[0] for x in top10]
feature_weights = [abs(x[1]) for x in top10]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(feature_names,feature_weights)
ax.set_title("Top 10 Features mas importantes segun XGBoost")
plt.xticks(rotation=30, ha='right')
plt.show()

## Conclusion

Segun LogisticRegression, los 10 features mas importantes son

| Feature | Weight|
| ---|---|
|('C14'|-1.1169994326139894)|
|('C2'| 0.940987827815535)|
| ('C11'| 0.748685001072886)|
 |('C13'| -0.6306327539310376)|
| ('C7'| -0.6034318025054934)|
| ('C3'| -0.6009501739533933)|
| ('V192'| 0.5754602374874863)|
| ('V222'| 0.4699256928563515)|
| ('V179'| 0.4553329097486659)|
| ('DeviceType'| 0.4431715326946056)|

Y segun XGBooster


| Feature | Weight|
| ---|---|
|('V257'| 0.17444326)|
|('V246'| 0.08324755)|
|('V189'| 0.060298882|
|('V200'| 0.044061814|
|('C7'| 0.029080976|
|('id_35'| 0.027788244| 
|('mra58k'| 0.027283499)| 
|('V188' |0.022855962| 
|('V149' | 0.021715455|
|('C11' | 0.02164322)|

Observamos que en ambas listas estan predominantes features de la categoria C y V. Posiblemente estas tengan una relacion importante por repetirse en ambos modelos.